In [6]:
import yfinance as yf
import tensorflow as tf
import numpy as np
import pandas as pd

tickers = ["AB.PA", "A2A.MI", "ABCA.PA", "ABNX.PA", "ABVX.PA", "AC.PA", "ADP.PA", "AIR.PA"]
data = yf.download(tickers, start='2022-01-01', end='2023-01-01')

def remove_top_column_name(df):
    return df.droplevel(0, axis=1)

close_prices = data[['Close']]
close_prices = remove_top_column_name(close_prices)

close_prices.head(5)
# Valeurs historiques des 10 actions
# close_prices = pd.DataFrame(
#     np.array([
#         [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
#         [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
#         ...
#     ]),
#     index=pd.to_datetime(['2023-01-01', '2023-01-02', '...']),
#     columns=['action_0', 'action_1', '...']
# )

# Nombre d'actions
n_actions = close_prices.shape[1]

[*********************100%***********************]  8 of 8 completed


In [10]:
# Créer un modèle RNN
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(n_actions, activation='softmax')  # Output layer for action predictions
])

# Initialiser les poids du modèle
model.build(input_shape=(None, close_prices.shape[1]))
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 128)               1152      
                                                                 
 dense_7 (Dense)             (None, 32)                4128      
                                                                 
 dense_8 (Dense)             (None, 8)                 264       
                                                                 
Total params: 5,544
Trainable params: 5,544
Non-trainable params: 0
_________________________________________________________________


In [11]:
# Fonction d'optimisation
def optimize(model, close_prices, initial_value):
    # Initialiser le portefeuille
    portfolio = np.zeros(n_actions)
    portfolio[0] = initial_value

    # Itérer sur les données
    for close_price in close_prices.iterrows():
        # Prédire la prochaine action
        action = model.predict(close_price[1].values[np.newaxis, Ellipsis])[0]

        # Effectuer l'action
        if action == 0:
            # Acheter
            portfolio[action] += close_price[1][action]
        elif action == 1:
            # Vendre
            portfolio[action] -= close_price[1][action]
        else:
            # Attendre
            pass

    # Retourner la valeur du portefeuille
    return portfolio.sum()

In [13]:
# Entraîner le modèle
for epoch in range(1000):
    # Générer un jeu de données d'entraînement
    close_prices_train = close_prices[:epoch]

    # Optimiser le modèle
    optimized_value = optimize(model, close_prices_train, 1000)

    # Afficher la valeur du portefeuille
    print(f"Epoch {epoch}: {optimized_value} €")

Epoch 0: 1000.0 €


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()